# Amplifierd API - Profile Management

This notebook demonstrates profile discovery and management operations.

## Overview

Profiles in amplifier define:
- Provider configurations (LLM services)
- Tool selections
- Hook configurations
- Orchestrator settings

This notebook covers both **read operations** (Phase 1) and **write operations** (Phase 2).

In [ ]:
import json

import requests

BASE_URL = "http://127.0.0.1:8420"
API_BASE = f"{BASE_URL}/api/v1"


def print_response(response: requests.Response, title: str = "") -> None:
    if title:
        print(f"\n{'=' * 60}")
        print(f"{title}")
        print(f"{'=' * 60}")
    print(f"Status: {response.status_code} {response.reason}")
    if response.content:
        try:
            data = response.json()
            print(json.dumps(data, indent=2))
            return data
        except json.JSONDecodeError:
            print(response.text)
            return None
    return None


print("✓ Setup complete")

## Phase 1: Read Operations

### List All Profiles

Get all available profiles:

In [ ]:
response = requests.get(f"{API_BASE}/profiles/")
profiles = print_response(response, "LIST PROFILES")

if profiles:
    print(f"\n✓ Found {len(profiles)} profile(s)")
    for profile in profiles:
        active = "[ACTIVE]" if profile.get("is_active") else ""
        print(f"  - {profile['name']} {active} (source: {profile['source']})")

### Get Active Profile

Retrieve the currently active profile:

In [ ]:
response = requests.get(f"{API_BASE}/profiles/active")
active_profile = print_response(response, "GET ACTIVE PROFILE")

if active_profile:
    print(f"\n✓ Active profile: {active_profile['name']}")

### Get Profile Details

Get detailed information about a specific profile:

In [ ]:
# Get details for the default profile
profile_name = "default"  # Change this to any available profile

response = requests.get(f"{API_BASE}/profiles/{profile_name}")
profile_details = print_response(response, f"GET PROFILE: {profile_name}")

if profile_details:
    print(f"\n✓ Profile: {profile_details['name']}")
    print(f"  Version: {profile_details.get('version', 'N/A')}")
    print(f"  Description: {profile_details.get('description', 'N/A')}")
    print(f"  Source: {profile_details['source']}")
    print(f"  Active: {profile_details['is_active']}")

    if profile_details.get("inheritance_chain"):
        print(f"  Inheritance: {' -> '.join(profile_details['inheritance_chain'])}")

    print(f"  Providers: {len(profile_details.get('providers', []))}")
    print(f"  Tools: {len(profile_details.get('tools', []))}")
    print(f"  Hooks: {len(profile_details.get('hooks', []))}")

### Explore Profile Components

Examine providers, tools, and hooks in a profile:

In [ ]:
if profile_details:
    print("Providers:")
    for provider in profile_details.get("providers", []):
        print(f"  - {provider['module']} (source: {provider.get('source', 'default')})")

    print("\nTools:")
    for tool in profile_details.get("tools", []):
        print(f"  - {tool['module']}")

    print("\nHooks:")
    for hook in profile_details.get("hooks", []):
        print(f"  - {hook['module']}")

## Phase 2: Write Operations

### Activate a Profile

Set a profile as active:

In [ ]:
# Activate a profile (use a profile name from the list above)
profile_to_activate = "default"

response = requests.post(f"{API_BASE}/profiles/{profile_to_activate}/activate")
result = print_response(response, f"ACTIVATE PROFILE: {profile_to_activate}")

if response.ok:
    print(f"\n✓ Activated profile: {result['name']}")
    print(f"  Status: {result['status']}")
elif response.status_code == 404:
    print(f"\n✗ Profile '{profile_to_activate}' not found")
else:
    print(f"\n✗ Failed to activate profile: {response.status_code}")

### Verify Profile Activation

Confirm the profile is now active:

In [ ]:
response = requests.get(f"{API_BASE}/profiles/active")
active = print_response(response, "VERIFY ACTIVATION")

if active:
    print(f"\n✓ Confirmed: '{active['name']}' is active")

### Deactivate Current Profile

Clear the active profile:

In [ ]:
response = requests.delete(f"{API_BASE}/profiles/active")
result = print_response(response, "DEACTIVATE PROFILE")

if response.ok:
    print("\n✓ Profile deactivated")

    # Verify no active profile
    response = requests.get(f"{API_BASE}/profiles/active")
    if response.status_code == 200:
        active = response.json()
        if not active:
            print("✓ Confirmed: No active profile")
    else:
        print("✓ Confirmed: No active profile")

### Error Handling: Non-Existent Profile

Attempt to activate a profile that doesn't exist:

In [ ]:
fake_profile = "non-existent-profile"
response = requests.post(f"{API_BASE}/profiles/{fake_profile}/activate")
print_response(response, f"ACTIVATE NON-EXISTENT: {fake_profile}")

if response.status_code == 404:
    print("\n✓ Correctly returns 404 for non-existent profile")

## Complete Profile Workflow

Demonstrate a full profile management workflow:

In [ ]:
def profile_workflow():
    """Complete profile management workflow."""

    # 1. List available profiles
    print("1. Listing profiles...")
    response = requests.get(f"{API_BASE}/profiles/")
    if not response.ok:
        print("✗ Failed to list profiles")
        return

    profiles = response.json()
    print(f"✓ Found {len(profiles)} profiles")

    # 2. Get details for first profile
    if profiles:
        profile_name = profiles[0]["name"]
        print(f"\n2. Getting details for '{profile_name}'...")
        response = requests.get(f"{API_BASE}/profiles/{profile_name}")
        if response.ok:
            details = response.json()
            print(f"✓ Profile has {len(details.get('providers', []))} providers")

        # 3. Activate the profile
        print(f"\n3. Activating '{profile_name}'...")
        response = requests.post(f"{API_BASE}/profiles/{profile_name}/activate")
        if response.ok:
            print("✓ Activated successfully")

        # 4. Verify activation
        print("\n4. Verifying activation...")
        response = requests.get(f"{API_BASE}/profiles/active")
        if response.ok:
            active = response.json()
            if active and active["name"] == profile_name:
                print(f"✓ Confirmed: '{profile_name}' is active")

    print("\n✓ Workflow complete")


profile_workflow()

## Summary

### Phase 1 Operations (Read-Only)
- ✓ List all available profiles
- ✓ Get active profile
- ✓ Get detailed profile information
- ✓ Explore profile components (providers, tools, hooks)

### Phase 2 Operations (Write)
- ✓ Activate a profile
- ✓ Deactivate current profile
- ✓ Error handling for non-existent profiles

## API Endpoints Reference

| Method | Endpoint | Description | Phase |
|--------|----------|-------------|-------|
| GET | `/api/v1/profiles/` | List all profiles | 1 |
| GET | `/api/v1/profiles/active` | Get active profile | 1 |
| GET | `/api/v1/profiles/{name}` | Get profile details | 1 |
| POST | `/api/v1/profiles/{name}/activate` | Activate profile | 2 |
| DELETE | `/api/v1/profiles/active` | Deactivate profile | 2 |

## Next Steps

Continue to:
- **04-collection-management.ipynb** - Collection management
- **05-module-management.ipynb** - Module configuration